# Dota API

The first step is to figure out how we can use the Dota API to get a bunch of historical games.



## Basic API Usage

We'll use this repository as a starting point: https://github.com/bryanveloso/dota2.py

It's a little old and unmaintained but so is the SteamAPI!

In [1]:
import os
import json
import time
import datetime
import requests

We need a Steam API Key. Go get it from: https://steamcommunity.com/dev/apikey

If you like living on the edge you can embed it directly in this notebook and pray that you never commit it to GitHub. Otherwise, export it from your `.bashrc` as `STEAM_API_KEY`.

In [2]:
if 'STEAM_API_KEY' not in os.environ:
  print("No API Key :(")
else:
  print("Found API Key.")
  STEAM_API_KEY = os.environ['STEAM_API_KEY']

Found API Key.


In [3]:
base_url = 'https://api.steampowered.com'

In [4]:
def __request(method, path, **kwargs):
  url = base_url + path
  kwargs.setdefault('params', dict()).update(key=STEAM_API_KEY)
  response = requests.request(method, url, **kwargs)
  return response.json()

In [5]:
def get_steam_id(vanity_name, **params):
    path = '/ISteamUser/ResolveVanityURL/v0001'
    params.update(vanityurl=vanity_name)
    response = __request('get', path, params=params).get('response')
    if response and "success" in response and response['success'] == 1:
        return response
    else:
        print("Error:", response)

In [6]:
response = get_steam_id("Permit")
print("Response:", response)
steam_id = response['steamid']

Response: {'steamid': '76561197990194935', 'success': 1}


Nice. So we can successfully hit the API and get my ID back. 

## Get Information About a Player

In [7]:
def get_player_summaries(steam_ids, **params):
  path = '/ISteamUser/GetPlayerSummaries/v0002'
  params.update(steamids=steam_ids)
  return __request('get', path, params=params)

In [8]:
response = get_player_summaries(steam_id)
print(response)

{'response': {'players': [{'steamid': '76561197990194935', 'communityvisibilitystate': 3, 'profilestate': 1, 'personaname': 'Permit', 'profileurl': 'https://steamcommunity.com/id/Permit/', 'avatar': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/fe/fef49e7fa7e1997310d705b2a6158ff8dc1cdfeb.jpg', 'avatarmedium': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/fe/fef49e7fa7e1997310d705b2a6158ff8dc1cdfeb_medium.jpg', 'avatarfull': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/fe/fef49e7fa7e1997310d705b2a6158ff8dc1cdfeb_full.jpg', 'avatarhash': 'fef49e7fa7e1997310d705b2a6158ff8dc1cdfeb', 'lastlogoff': 1627876474, 'personastate': 0, 'realname': 'Josh', 'primaryclanid': '103582791429988448', 'timecreated': 1180666476, 'personastateflags': 0, 'loccountrycode': 'CA'}]}}


## Get Information About Games

Let's start by getting the most recent games available.

In [9]:
def get_most_recent_matches():
  path = '/IDOTA2Match_570/GetMatchHistory/V001'
  return __request('get', path)

In [10]:
response = get_most_recent_matches()
response

{'result': {'status': 1,
  'num_results': 100,
  'total_results': 500,
  'results_remaining': 400,
  'matches': [{'match_id': 6116539419,
    'match_seq_num': 5119431667,
    'start_time': 1627882306,
    'lobby_type': 4,
    'radiant_team_id': 0,
    'dire_team_id': 0,
    'players': [{'account_id': 4294967295, 'player_slot': 128, 'hero_id': 0},
     {'player_slot': 0, 'hero_id': 0},
     {'player_slot': 1, 'hero_id': 0},
     {'player_slot': 2, 'hero_id': 0},
     {'player_slot': 3, 'hero_id': 0},
     {'player_slot': 4, 'hero_id': 0},
     {'player_slot': 129, 'hero_id': 0},
     {'player_slot': 130, 'hero_id': 0},
     {'player_slot': 131, 'hero_id': 0},
     {'player_slot': 132, 'hero_id': 0}]},
   {'match_id': 6116538920,
    'match_seq_num': 5119431252,
    'start_time': 1627882275,
    'lobby_type': 14,
    'radiant_team_id': 0,
    'dire_team_id': 0,
    'players': [{'account_id': 4294967295, 'player_slot': 128, 'hero_id': 0},
     {'player_slot': 0, 'hero_id': 0},
     {'play

Great! It looks like the games are still in progress though. Some of them don't appear to have finished hero selection so they won't be very useful for us. 

Let's try getting older games.

In [11]:
def get_matches_before(match_id, **params):
  path = '/IDOTA2Match_570/GetMatchHistory/V001'
  params.update(start_at_match_id=match_id)
  return __request('get', path, params=params)

In [12]:
response = get_matches_before(6116424555)
response

{'result': {'status': 1,
  'num_results': 0,
  'total_results': 500,
  'results_remaining': 0,
  'matches': []}}

Hmm this isn't working. Maybe it limits it to the most recent 500 games? That would suck. I will revisit this later.